# ML similitud del coseno

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

## EDA preliminar

En primer lugar se realizará un análisis exploratorio para poder reducir los datos debido a los recursos del entorno donde se hace el despliegue de la aplicación 

In [2]:
#Se cargan los datos
df_games=pd.read_parquet('Datasets/data_ML.parquet')
df_users=pd.read_parquet('Datasets/users_items_clean.parquet')
df_games['release_year']=df_games['release_year'].astype('int16')

In [3]:
#Se unen los datasets
df_most_popular=pd.merge(df_games,df_users[['item_id','playtime_forever']],left_on='id',right_on='item_id',how='inner')
#Se eliminan las columnas que no se necesitan
df_most_popular.drop(columns=['item_id','id'],inplace=True)

Se filtran los juegos por los que se lanzaron a partir del 2014 para considerar en las recomendaciones los juegos mas nuevos.

In [4]:
#Se filtra el dataframe por los juegos lanzados a partir del 2014
df_most_popular=df_most_popular[df_most_popular['release_year']>=2014]
#Se agrupa por el nombre del juego y se suma el tiempo de juego
df_playtime=df_most_popular.groupby('app_name')['playtime_forever'].sum().reset_index()
#Se ordenan los juegos por tiempo de juego
df_playtime.sort_values(by='playtime_forever',ascending=False)
#Se eleimina los juegos con tiempo de juego igual a 0
df_playtime=df_playtime[df_playtime['playtime_forever']>0]
#Se unen los datasets
data=pd.merge(df_playtime['app_name'],df_games,left_on='app_name',right_on='app_name',how='inner')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5211 entries, 0 to 5210
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   app_name      5211 non-null   object
 1   genres        5211 non-null   object
 2   tags          5211 non-null   object
 3   id            5211 non-null   int32 
 4   release_year  5211 non-null   int16 
dtypes: int16(1), int32(1), object(3)
memory usage: 152.8+ KB


In [6]:
#Se elimina la columna del año de lanzamiento
data.drop('release_year',axis=1,inplace=True)

#Se guardan los datos en un archivo parquet
data.to_parquet('Datasets/data.parquet')

## Modelo de la similitud del coseno

In [18]:
df=pd.read_parquet('Datasets/data.parquet')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5211 entries, 0 to 5210
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   app_name  5211 non-null   object
 1   genres    5211 non-null   object
 2   tags      5211 non-null   object
 3   id        5211 non-null   int32 
dtypes: int32(1), object(3)
memory usage: 142.6+ KB


In [20]:
df.reset_index(drop=True, inplace=True)

In [21]:
#Vectorizar el nombre de los productos
vectorizer=TfidfVectorizer()
tfidf_matrix=vectorizer.fit_transform(df[['app_name','genres','tags']].apply(lambda x: ' '.join(x),axis=1))

In [22]:
# Calcular la cantidad de memoria en bytes
memory_in_bytes = (tfidf_matrix.data.nbytes + tfidf_matrix.indices.nbytes + tfidf_matrix.indptr.nbytes)

# Convertir a megabytes
memory_in_megabytes = memory_in_bytes / (1024 * 1024)

print(f"Memory: {memory_in_megabytes} MB")

Memory: 0.2952156066894531 MB


In [24]:
#Calcular la matriz de similitud del coseno
similarity_matrix=cosine_similarity(tfidf_matrix)

In [25]:
# Calcular la cantidad de memoria en bytes
memory_in_bytes = similarity_matrix.nbytes

# Convertir a megabytes
memory_in_megabytes = memory_in_bytes / (1024 * 1024)

print(f"Memory: {memory_in_megabytes} MB")

Memory: 207.17255401611328 MB


In [27]:
#Se crea el modelo de recomendación y se prueba con un id de un juego
item_id=508290
if df[df['id']==item_id].empty:
    print('No existe el item')
else:
    producto_index=df[df['id']==item_id].index[0]
    product_similarities=similarity_matrix[producto_index]
    most_similar_products_index=np.argsort(-product_similarities)[1:6]
    most_similar_products=df.loc[most_similar_products_index,'app_name']
    print(most_similar_products)

3592             Ride The Bullet
3591                        Ride
3925           Slide Ride Arcade
1764    Funfair Ride Simulator 3
3458                   R.O.O.T.S
Name: app_name, dtype: object


In [28]:
#Se revisan los resultados
most_similar_products.values.tolist()

['Ride The Bullet',
 'Ride',
 'Slide Ride Arcade',
 'Funfair Ride Simulator 3',
 'R.O.O.T.S']